In [373]:
import pandas as pd
import numpy as np
import re
from estnltk import Text
from estnltk_neural.taggers import StanzaSyntaxTagger

In [311]:
df = pd.read_excel('data/12_eesti_ilmik_rektsiooniga_linkideta.xlsx')

In [312]:
df

,wordid,lexeme_id,word,homonym_nr,is_public,pos,government
0,154548,1141002,aasima,1,True,v,keda*
1,154548,1141002,aasima,1,True,v,kelle kallal
2,154678,1141297,abielluma,1,True,v,kellega
3,154740,1141437,abikätt ulatama,1,True,v,kellele
4,154772,1141513,abistama,1,True,v,keda/mida* + milles
...,...,...,...,...,...,...,...
3319,264050,1413420,ümbritsema,1,True,v,millega
3320,264058,1413464,ümmardama,1,True,v,keda*
3321,264120,1413609,üritama,1,True,v,mida teha
3322,264132,1413655,ütlema,1,True,v,kelle/mille kohta


### Creating verb + word case patterns

#### I Mapping questions to cases

In [27]:
display(df['pos'].unique())

array(['v', 'adv,postp', 'postp', nan, 's', 's,v', 'vrm', 'adv',
       'adv,prep', 'prep', 'adj'], dtype=object)

In [308]:
# mapping 14 + 1 cases to their corresponding questions

#ab	abessiiv (ilmaütlev)
#abl	ablatiiv (alaltütlev)
#ad	adessiiv (alalütlev)
#adt	aditiiv suunduv (lühike sisseütlev)
#all	allatiiv (alaleütlev)
#el	elatiiv (seestütlev)
#es	essiiv (olev)
#g	genitiiv (omastav)
#ill	illatiiv (sisseütlev)
#in	inessiiv (seesütlev)
#kom	komitatiiv (kaasaütlev)
#n	nominatiiv (nimetav)
#p	partitiiv (osastav)
#ter	terminatiiv (rajav)
#tr	translatiiv (saav)

case_question_mapping = {'nom': ['kes', 'mis', 'milline'],
                   'g': ['kelle', 'mille', 'millise'],
                   'p': ['keda', 'mida', 'millist'],
                   'ill': ['kellesse', 'millesse', 'millisesse'],
                   'adt': ['kuhu'],
                   'in': ['kelles', 'milles', 'kus', 'millises'],
                   'el': ['kellest', 'millest', 'kust', 'millisest'],
                   'all': ['kellele', 'millele', 'kuhu', 'millisele'],
                   'ad': ['kellel', 'millel', 'kus', 'millisel'],
                   'abl': ['kellelt', 'millelt', 'kust', 'milliselt'],
                   'tr': ['kelleks', 'milleks', 'milliseks'],
                   'ter': ['kelleni', 'milleni', 'milliseni'],
                   'es': ['kellena', 'millena', 'millisena'],
                   'ab': ['kelleta', 'milleta', 'milliseta'],
                   'kom': ['kellega', 'millega', 'millisega']}


question_case_mapping = {'kes': 'nom', 'mis': 'nom', 'milline': 'nom',
                   'kelle': 'g', 'mille': 'g', 'millise': 'g',
                   'keda': 'p', 'mida': 'p', 'millist': 'p',
                   'kellesse': 'ill', 'millesse': 'ill', 'millisesse': 'ill',
                   'kuhu': 'adt',
                   'kelles': 'in', 'milles': 'in', 'kus': 'in', 'millises': 'in',
                   'kellest': 'el', 'millest': 'el', 'kust': 'el', 'millisest': 'el',
                   'kellele': 'all', 'millele': 'all', 'kuhu': 'all', 'millisele': 'all',
                   'kellel': 'ad', 'millel': 'ad', 'kus': 'ad', 'millisel': 'ad',
                   'kellelt': 'abl', 'millelt': 'abl', 'kust': 'abl', 'milliselt': 'abl',
                   'kelleks': 'tr', 'milleks': 'tr', 'milliseks': 'tr',
                   'kelleni': 'ter', 'milleni': 'ter', 'milliseni': 'ter',
                   'kellena': 'es', 'millena': 'es', 'millisena': 'es',
                   'kelleta': 'ab', 'milleta': 'ab', 'milliseta': 'ab',
                   'kellega': 'kom', 'millega': 'kom', 'millisega': 'kom'}



In [ ]:
# Tähelepanekud veeru 'government' kohta:
#
# -- '/' tähendab üks või teine, nt mille/kelle
# -- '*' tähendust ei tea
# -- ' + ' käib pikema konstruktsiooni liikmete vahele (nt kääne + kaassõna, kääne koos kaassõnaga + kääne, kääne + kääne vms)
# -- alati pole käändeinfot, vahel on lihtsalt kaassõna, sidesõna vms
# -- tühikuga on eraldatud ühe terviku osad
#
# Tähelepanekud veeru 'pos' kohta:
#
# -- valdavalt verbid
# -- kohati s, adv, adj, prep, postp, vrm
# -- korraga võimalik rohkem, kui üks väärtus
# -- kohati on lahter tühi. Tavaliselt on siiski eelmises/järgnevas reas identne näide.
#
#
# Lähenemine:
#
# -- võtta read, mille 'pos' väärtuste hulgas on 'v', samuti võtta tühjad read
# -- ülejäänud ridu hetkel ignoreerida
# -- eraldada küsisõna ja muud liikmed, küsisõna puudumisel ainult muu liige. Küsisõna asemele kääne
# -- hetkel ignoreerida tärni (*)
# -- panna kokku verb + kääne (ja muud konstruktsiooni liikmed, kui on)

In [ ]:
# -- võtta read, mille 'pos' väärtuste hulgas on 'v', samuti võtta tühjad read

In [139]:
display(df['pos'].unique())

array(['v', 'adv,postp', 'postp', nan, 's', 's,v', 'vrm', 'adv',
       'adv,prep', 'prep', 'adj'], dtype=object)

In [33]:
df_verbs = df[(df['pos'] == 's,v') | (df['pos'] == 'v') | (pd.isna(df['pos']))]

In [34]:
df_verbs

,wordid,lexeme_id,word,homonym_nr,is_public,pos,government
0,154548,1141002,aasima,1,True,v,keda*
1,154548,1141002,aasima,1,True,v,kelle kallal
2,154678,1141297,abielluma,1,True,v,kellega
3,154740,1141437,abikätt ulatama,1,True,v,kellele
4,154772,1141513,abistama,1,True,v,keda/mida* + milles
...,...,...,...,...,...,...,...
3319,264050,1413420,ümbritsema,1,True,v,millega
3320,264058,1413464,ümmardama,1,True,v,keda*
3321,264120,1413609,üritama,1,True,v,mida teha
3322,264132,1413655,ütlema,1,True,v,kelle/mille kohta


In [35]:
display(df_verbs['pos'].unique())

array(['v', nan, 's,v'], dtype=object)

In [36]:
# -- eraldada küsisõna ja muud liikmed, küsisõna puudumisel ainult muu liige. Küsisõna asemele kääne

In [129]:
# test kombinatsioonid
test1 = 'kelle käest + mida*'
test2 = 'kelle/mille kohta'
test3 = 'kelle/mida kohta'
test4 = 'millele / mille peale'
test5 = 'mis'

In [137]:
def get_mapping(text, mapping_dict):
    if text in mapping_dict.keys():
        return question_case_mapping[text]
    return text

def split_members(text):
    cleaned_text = re.sub('\*', '', text) # removing asterisks
    splitted = cleaned_text.split(' + ') # splitting members
    
    for i in range(len(splitted)):
        splitted[i] = splitted[i].split() # splitting members by whitespaces          
        
        for j in range(len(splitted[i])):
            if '/' in splitted[i][j] and splitted[i][j] != '/':
                splitted[i][j] = splitted[i][j].split('/') # splitting alternatives
            
                for k in range(len(splitted[i][j])): # substituting question words with corresponding cases
                    splitted[i][j][k] = get_mapping(splitted[i][j][k], question_case_mapping)
                    
                splitted[i][j] = set(splitted[i][j]) # if alternatives represent same case, repetitions are removed
                splitted[i][j] = ' '.join(splitted[i][j])
                
            else:
                splitted[i][j] = get_mapping(splitted[i][j], question_case_mapping)

        splitted[i] = ' '.join(splitted[i])
        
    return ' + '.join(splitted)

In [138]:
print(split_members(test1))
print(split_members(test2))
print(split_members(test3))
print(split_members(test4))
print(split_members(test5))

g käest + p
g kohta
g p kohta
all / g peale
nom


In [ ]:
# Tulemus:
#
# -- ' + ' eraldab pikema konstruktsiooni eri liikmeid
# -- ' / ' eraldab eri võimalikke variante
# -- ' ' käib sama liikme eri juppide (sõnade) vahele (nt kääne koos kaassõnaga)
#
# NB! Esialgu lisan käänetega mustrid kõigile rektsioonitabeli ridadele, sõltumata sõnaliigist

In [313]:
patterns_with_case = []

for idx, row in df.iterrows():
    patterns_with_case.append(split_members(row['government']))

In [314]:
len(patterns_with_case)

3324

In [315]:
df['government_with_case'] = patterns_with_case

In [316]:
df

,wordid,lexeme_id,word,homonym_nr,is_public,pos,government,government_with_case
0,154548,1141002,aasima,1,True,v,keda*,p
1,154548,1141002,aasima,1,True,v,kelle kallal,g kallal
2,154678,1141297,abielluma,1,True,v,kellega,kom
3,154740,1141437,abikätt ulatama,1,True,v,kellele,all
4,154772,1141513,abistama,1,True,v,keda/mida* + milles,p + in
...,...,...,...,...,...,...,...,...
3319,264050,1413420,ümbritsema,1,True,v,millega,kom
3320,264058,1413464,ümmardama,1,True,v,keda*,p
3321,264120,1413609,üritama,1,True,v,mida teha,p teha
3322,264132,1413655,ütlema,1,True,v,kelle/mille kohta,g kohta


In [319]:
df.to_csv('data/rektsioon_käänetega.csv', encoding='utf-8')

In [320]:
# salvestan ka verb + muster eraldi
df2 = df[['word', 'government_with_case']].copy()
df2

,word,government_with_case
0,aasima,p
1,aasima,g kallal
2,abielluma,kom
3,abikätt ulatama,all
4,abistama,p + in
...,...,...
3319,ümbritsema,kom
3320,ümmardama,p
3321,üritama,p teha
3322,ütlema,g kohta


In [321]:
df2.to_csv('data/verb_patterns.csv', encoding='utf-8')

In [322]:
df2

,word,government_with_case
0,aasima,p
1,aasima,g kallal
2,abielluma,kom
3,abikätt ulatama,all
4,abistama,p + in
...,...,...
3319,ümbritsema,kom
3320,ümmardama,p
3321,üritama,p teha
3322,ütlema,g kohta


In [323]:
for idx, row in df2.iterrows():
    if '+' in row['government_with_case']:
        print(row['government_with_case'])

p + in
ad + p teha
all + p
p + el
p + kom
ad + p teha
all + g peale
p + kuidas
kom + in
ad + p teha
g käest + p
abl + p
ad + p teha
ad + p teha
ad + p teha
kom + g suhtes
ad + tr
ad + p teha
p + tr
p + tr
all + el
all + p
ad + p teha
p + p tegema
ad + p teha
p + el
abl + all
all + kom
ad + p teha
nom + es
nom keelest + nom keelde
p + kom
all + kom
abl + all
ad + p teha


In [324]:
# alternatiivsed variandid on eraldatud kaldkriipsuga
count = 0
for idx, row in df2.iterrows():
    if '/' in row['government_with_case']:
        print(row['government_with_case'])
        count += 1
print(count)

abl / g käest
tr / p teha
el / g kohta
g juurde / g kallale
g tõttu / g tagajärjel
g pärast / g tõttu
g peale / g eest
all / g peale
el / g eest
abl / el
all / g peale
g eest / kom
ad / in
g eest / kom
abl / g käest
abl / abl
vastu p / g vastu
el / g hulgast
kom / g vastu
ad / g juures
20


In [325]:
koopia = df2.copy()

In [326]:
len(koopia)

3324

In [327]:
# alternatiivid lüüakse lahku ning sisestatakse dataframe-i eraldi ridadele
i = 0
size = len(koopia)
pattern = r'(\D+) / (\D+)'
while i < size-1:
    if '/' in koopia.loc[i]['government_with_case']:
        row_to_add = koopia.loc[i].copy()
        row_to_add['government_with_case'] = re.sub(pattern, r'\2', koopia.loc[i]['government_with_case'])
        koopia.loc[i]['government_with_case'] = re.sub(pattern, r'\1', koopia.loc[i]['government_with_case'])
        print(koopia.loc[i]['government_with_case'])
        print(row_to_add['government_with_case'])
        print()
        koopia.loc[(i+1)/i] = row_to_add
        koopia = koopia.sort_index().reset_index(drop=True)
        i+=2
        size=len(koopia)
    else:
        i+=1

abl
g käest

tr
p teha

el
g kohta

g juurde
g kallale

g tõttu
g tagajärjel

g pärast
g tõttu

g peale
g eest

all
g peale

el
g eest

abl
el

all
g peale

g eest
kom

ad
in

g eest
kom

abl
g käest

abl
abl

vastu p
g vastu

el
g hulgast

kom
g vastu

ad
g juures



In [328]:
# kontroll
count = 0
for idx, row in koopia.iterrows():
    if '/' in row['government_with_case']:
        print(row['government_with_case'])
        count += 1
print(count)

0


In [329]:
koopia = koopia.sort_values('word').reset_index(drop=True)

In [333]:
koopia = koopia.drop_duplicates()

In [334]:
koopia

,word,government_with_case
0,aasima,p
1,aasima,g kallal
2,abielluma,kom
3,abikätt ulatama,all
4,abistama,p + in
...,...,...
3338,ümmardama,p
3339,üritama,p teha
3340,ütlema,all
3341,ütlema,g kohta


In [415]:
# mustris üks liige, mis koosneb ühest sõnast, kuid mis ei ole kääne: põhiliselt sidesõnad, aja- või viisimäärused
# üks kord esinevad erandid: küsisõna 'kas', venekeelne sõna 'tšto' - 'mis', kaasaütlev kääne koos gi-liitega 'kellegagi' 
for idx, row in koopia.iterrows():
    if len(row['government_with_case'].split()) == 1:
        has_no_case = True
        for piece in row['government_with_case'].split():
            if piece in case_question_mapping.keys():
                has_no_case = False
        if has_no_case:
            print(row['word'], row['government_with_case'].split())

arvama ['et']
deklareerima ['et']
demonstreerima ['et']
eeldama ['et']
ette kujutama ['kuidas']
ette kujutama ['et']
hakkama ['millal']
hoolitsema ['et']
hoolt kandma ['et']
kaebama ['et']
kahetsema ['et']
kahtlema ['kas']
kahtlustama ['et']
kartma ['et']
kehtestama ['millal']
kinnitama ['et']
kogema ['et']
kokku leppima ['et']
kordama ['et']
kulgema ['kuidas']
kurtma ['et']
kuulma ['et']
käituma ['kuidas']
küsima ['kas']
leidma ['et']
lootma ['et']
märkama ['et']
märkima ['et']
näima ['et']
näitama ['et']
oletama ['et']
otsustama ['et']
pahandama ['et']
paistma ['et']
peale hakkama ['millal']
rääkima ['et']
rõhutama ['et']
süüdistama ['et']
tajuma ['et']
teatama ['et']
teesklema ['et']
toimima ['kuidas']
tundma ['kuidas']
tunduma ['et']
tunnetama ['et']
tunnistama ['et']
tuvastama ['et']
tähele panema ['et']
tähendama ['et']
täpsustama ['kas']
tõdema ['et']
tõestama ['et']
unistama ['et']
uskuma ['et']
vabandama ['et']
valima ['kas']
veenduma ['et']
vihjama ['et']
väitma ['et']
võtma 

In [380]:
# eraldatakse muud erandlikud juhud: juhud, kus ühene muster koosneb mitmest sõnast, mille seas puudub kään
for idx, row in koopia.iterrows():
    if len(row['government_with_case'].split()) > 1:
        has_no_case = True
        for piece in row['government_with_case'].split():
            if piece in case_question_mapping.keys():
                has_no_case = False
        if has_no_case:
            print(row['word'], row['government_with_case'].split())

kestma ['kui', 'kaua']
tuhnima ['в', 'чём']


In [339]:
# erandlik juht, kus üheliikmeline koosneb rohkem, kui kahest sõnast. Edaspidi siiski käsitletakse
# tavalise üheliikmelise mustrina
for idx, row in koopia.iterrows():
    if len(row['government_with_case'].split()) > 2 and '+' not in row['government_with_case']:
        print(row['word'], row['government_with_case'].split())

teada saama ['el', 'g', 'kohta']


In [390]:
# kui verbi mustrisd on üks liige (võib koosneda rohkem, kui ühest sõnast (siinsetes andmetes kuni 3 sõnast), 
# kuid need sõnad moodustavad ühe terviku)
#
# Kui sõna on kääne, siis ei lisandu lõppu täiendavat infot
# Kui sõna on verb, siis lisandub lõppu '.v'
# Kui sõna on osa erandlikust konstruktsioonist, kus käänet pole (siinsetes andmetes kaks juhtu), siis lisandub lõppu '.?'
# Muudel juhtudel lisandub lõppu '.adp', sest oletatavasti on tegemist kaassõnaga
#

def process_len1(pattern_str):
    pieces = pattern_str.split()
    member_dict = {'wrd_1': '', 'wrd_2': '', 'wrd_3': ''}
    has_no_case = True
    
    for i in range(len(pieces)):
        member_dict[f'wrd_{i+1}'] = pieces[i]
        if pieces[i] in case_question_mapping.keys():
            has_no_case = False
            
    for key in member_dict.keys():
        if member_dict[key] == '' or member_dict[key] in case_question_mapping.keys():
            continue
        else:
            if has_no_case:
                member_dict[key] = member_dict[key]+'.?'
            elif 'V' in Text(member_dict[key]).tag_layer('morph_analysis').morph_analysis.partofspeech[0]:
                member_dict[key] = member_dict[key]+'.v'
            else:
                member_dict[key] = member_dict[key]+'.adp'
    
    return member_dict

In [418]:
for idx, row in koopia.iterrows():
    if '+' not in row['government_with_case']:
        #print(row['government_with_case'])
        print(process_len1(row['government_with_case']))
        #print()
        break

{'wrd_1': 'p', 'wrd_2': '', 'wrd_3': ''}


In [400]:
wrd_1 = []
wrd_2 = []
wrd_3 = []
multiple_members = [] # indexes of dataframe rows

new_df1 = koopia.copy()

for idx, row in koopia.iterrows():
    if '+' in row['government_with_case']:
        multiple_members.append(idx)
        new_df1 = new_df1.drop(idx)
    else:
        member_dict = process_len1(row['government_with_case'])
        wrd_1.append(member_dict['wrd_1'])
        wrd_2.append(member_dict['wrd_2'])
        wrd_3.append(member_dict['wrd_3'])
        
new_df1['word_1'] = wrd_1
new_df1['word_2'] = wrd_2
new_df1['word_3'] = wrd_3

new_df1 = new_df1.reset_index(drop=True)

In [411]:
new_df2 = koopia.loc[multiple_members]
new_df2 = new_df2.reset_index(drop=True)

In [408]:
new_df1

,word,government_with_case,word_1,word_2,word_3
0,aasima,p,p,,
1,aasima,g kallal,g,kallal.adp,
2,abielluma,kom,kom,,
3,abikätt ulatama,all,all,,
4,abstraheeruma,el,el,,
...,...,...,...,...,...
2533,ümmardama,p,p,,
2534,üritama,p teha,p,teha.v,
2535,ütlema,all,all,,
2536,ütlema,g kohta,g,kohta.adp,


In [413]:
# saving to CSV-file
new_df1.to_csv('data/verb_patterns_len1.csv', encoding='utf-8')

In [412]:
new_df2

,word,government_with_case
0,abistama,p + in
1,aitama,ad + p teha
2,ette heitma,all + p
3,informeerima,p + el
4,jagama,p + kom
5,keelama,ad + p teha
6,kleepima,all + g peale
7,kohtlema,p + kuidas
8,kokku leppima,kom + in
9,käskima,ad + p teha


In [419]:
# kuna kaheliikmelisi on andmetes vaid 33, tehakse edasised täiendused manuaalselt
# saving to CSV-file
new_df2.to_csv('data/verb_patterns_len2.csv', encoding='utf-8')

In [424]:
new_df2_new = pd.read_csv('data/verb_patterns_len2.csv')

In [425]:
new_df2_new

,Unnamed: 0,word,government_with_case,member1_wrd1,member1_wrd2,member2_wrd1,member2_wrd2
0,0,abistama,p + in,p,NaN,in,NaN
1,1,aitama,ad + p teha,ad,NaN,p,teha.v
2,2,ette heitma,all + p,all,NaN,p,NaN
3,3,informeerima,p + el,p,NaN,el,NaN
4,4,jagama,p + kom,p,NaN,kom,NaN
5,5,keelama,ad + p teha,ad,NaN,p,teha.v
6,6,kleepima,all + g peale,all,NaN,g,peale.adp
7,7,kohtlema,p + kuidas,p,NaN,kuidas.?,NaN
8,8,kokku leppima,kom + in,kom,NaN,in,NaN
9,9,käskima,ad + p teha,ad,NaN,p,teha.v
